# Fake News Detection Using Machine Learning


**About the Dataset:**

**train.csv:** A full training dataset with the following attributes:

**1.** id: unique id for a news article  
**2.** title: the title of a news article  
**3.** author: author of the news article  
**4.** text: the text of the article; could be incomplete  
**5.** label: a label that marks whether the news article is real or fake:

**1:** Fake news  
**0:** real News


## Importing Libraries


In [1]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

The initial phase of the project involves data pre- processing, where we delve into the implementation and usage of various Python-built modules. This approach helps us comprehend the significance of employing pre-existing modules, offering a more efficient and understandable code representation than developing functions from scratch. Noteworthy libraries utilized in this process include numpy, re, pandas, nltk, sklearn, matplotlib and more.

Data preprocessing includes removing stopwords that have no significance in our purpose, then creating datafame of our dataset, counting and replacing the null values in the dataset with empty string, performing stemming to get the root words, separating the data and label, vectorizing the textual data into numerical data, splitting the dataset into training and testing dataset to train the model.

For model training, initially we have used logistic regression since it is best suited for binary class classification problems. After training we have calculated accuracy on training as well as testing dataset and then created classification report and confusion matrix using sklearn library.


In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\vivek
[nltk_data]     singh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
## printing the stopwords in English
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

## loading the dataset to a pandas DataFrame


In [4]:
news_dataset = pd.read_csv('train.csv')

In [5]:
news_dataset.shape


(20800, 5)

In [6]:
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


## Data Pre-processing


**_Counting the number of missing values in the dataset_**


In [7]:
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

**_replacing the null values with empty string_**


In [8]:
news_dataset = news_dataset.fillna('')

**_merging the author name and news title_**


In [9]:
news_dataset['content'] = news_dataset['author']+' '+ news_dataset['title']

In [10]:
print(news_dataset['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


**_separating the data & label_**


In [11]:
X = news_dataset.drop(columns='label', axis=1)
Y = news_dataset['label']
print(X)
print(Y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

### Stemming:

_Stemming is the process of reducing a word to its Root word_

**example:** actor, actress, acting --> act


In [12]:
port_stem = PorterStemmer()

In [13]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower().split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [ ]:
news_dataset['content'] = news_dataset['content'].apply(stemming)
print(news_dataset['content'])

**_separating the data and label_**


In [ ]:
X = news_dataset['content'].values
Y = news_dataset['label'].values
print(X)
print(Y)

**_converting the textual data to numerical data_**


In [ ]:
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [ ]:
print(X)

**_Splitting the dataset to training & test data_**


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=2)

## Training the Model: Logistic Regression


In [ ]:
model = LogisticRegression()

In [ ]:
model.fit(X_train, Y_train)

### Evaluation


### accuracy score


**_accuracy score on the training data_**|


In [ ]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)
print('Accuracy score of the training data : ', training_data_accuracy)

accuracy score on the test data


In [ ]:
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)
print('Accuracy score of the test data : ', test_data_accuracy)

## Classification Report


In [ ]:
# Predictions on test data
X_test_prediction = model.predict(X_test)

# Generate classification report
report = classification_report(Y_test, X_test_prediction)
print(report)

## Confusion Matrix


In [ ]:
# Generate confusion matrix
cm = confusion_matrix(Y_test, X_test_prediction)

# Display confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap='Blues')

# Show plot
plt.title('Confusion Matrix')
plt.show()

### Visual Meaning:

Diagonal elements (TN and TP): Correct predictions.  
Off-diagonal elements (FP and FN): Incorrect predictions.


## Making a Predictive System


In [ ]:
X_new = X_test[3]

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

In [ ]:
print(Y_test[2])